In [2]:
from bs4 import BeautifulSoup as bs
import requests

In [361]:
rq = requests.get('https://pt.wikipedia.org/wiki/Fright_Night_(2011)')

soup = bs(rq.content, 'lxml')

contents = soup.prettify()

info_box = soup.find(class_ = 'infobox infobox_v2')

In [449]:
def scrape_movies():
    
    r = requests.get('https://pt.wikipedia.org/wiki/Lista_de_filmes_da_Disney')
    soup = bs(r.content, 'lxml')
    disney = []
    for decade in soup.find_all('table', class_='wikitable')[1:]:
        print('Next Decade...')
        for movie in decade.select('tbody td a'):
            partial_link = movie['href']
            title = movie['title']
            url = 'https://pt.wikipedia.org'+partial_link
            disney.append(movie_details(url))
            
    return disney


In [445]:
def get_content_value(row_data):
    
    if (len(row_data.find_all(text=True)) <= 2 and row_data.find_all(text=True)[-1] == '\n') or (len(row_data.find_all(text=True)) == 1):
        return row_data.get_text().strip('\n')
                
    elif len(row_data.find_all(text=True)) >= 2:
        return [li.get_text().strip('\n') for li in row_data.find_all(text=True) if len(li.get_text())>1]


def movie_details(url):

    rq = requests.get(url)

    soup = bs(rq.content, 'lxml')

    contents = soup.prettify()

    info_box = soup.find(class_ = 'infobox infobox_v2')
    
    movie_info = {}
    flag = False
    if info_box is not None:
        for index, row in enumerate(info_box.find_all("tr")):

            if index == 0:
                movie_info['Título'] = row.find('th').span.get_text()

            elif index == 1:
                continue

            elif row.get_text().strip('\n') == 'Pôster promocional.':
                continue

            elif row.find('th') and row.get_text().strip('\n') != 'Cronologia':
                if row.find_all('a'):
                    movie_info['País'] = row.find_all('a')[0].get_text()
                else:
                    movie_info['País'] = 'N/A'
                if len(row.find_all('a')) > 1:
                    movie_info['Ano'] = row.find_all('a')[1].get_text()
                else:
                    movie_info['Ano'] = 'N/A'
                flag = True

            elif row.get_text().strip('\n') == 'Cronologia':
                break

            elif flag:
                content_key = row.find_all('td')[0].get_text().strip('\n')
                if len(row.find_all('td')) > 1:
                    content_value = get_content_value(row.find_all('td')[1])
                else:
                    content_value = 'N/A'
                movie_info[content_key] = content_value
    else:
        return False
        
    return movie_info

        

In [448]:
import json 

In [452]:
with open('disney.json','w') as jdisney:
    json.dump(disney, jdisney)

In [453]:
disney

[{'Título': 'Academy Award Review of Walt Disney Cartoons'},
 {'Título': 'Branca de Neve e os Sete Anões',
  'País': 'Estados Unidos',
  'Ano': '1937',
  'Direção': 'David Hand',
  'Produção': 'Walt Disney',
  'Roteiro': ['Dorothy Ann Blank',
   'Richard Creedon',
   'Merrill De Maris',
   'Ted Sears',
   'Otto Englander',
   'Earl Hurd',
   'Dick Rickard',
   'Webb Smith'],
  'Baseado em': ['Branca de Neve', 'dos ', 'Irmãos Grimm'],
  'Elenco': ['Adriana Caselotti',
   'Harry Stockwell',
   'Lucille La Verne',
   'Moroni Olsen',
   'Billy Gilbert',
   'Pinto Colvig',
   'Eddie Collins',
   'Otis Harlan',
   'Scotty Mattraw',
   'Roy Atwell',
   'Stuart Buchanan'],
  'Gênero': ['filme musical', 'filme de animação', 'filme de fantasia'],
  'Música': ['Frank Churchill', 'Paul Smith', 'Leigh Harline'],
  'Companhia(s) produtora(s)': 'Walt Disney Productions',
  'Distribuição': 'RKO Radio Pictures',
  'Lançamento': ['EUA',
   ' 21 de dezembro de 1937',
   'BRA',
   ' 10 de janeiro de 1938'